In [4]:
import sys
datasetName = sys.argv[1] #'airports' # 'hospital', 'ncvoter', 'inspection'
result_file = sys.argv[2]
cudaOption = sys.argv[3]
train_round = sys.argv[4]
train_ratio = sys.argv[5]

import pandas as pd
import numpy as np
import os

data_prefix = '../../REEs_model_data/revision/labeled_data_400/'
# option = 'train'
# rule_pairs_train_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
# option = 'valid'
# rule_pairs_valid_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
# option = 'test'
# rule_pairs_test_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')

saved_model_path = os.path.join("./saved_models/", datasetName + '-transformer-bc')


In [ ]:
# extract rules from rule_pair_ids

rules_set_path = os.path.join(data_prefix, datasetName, 'train', 'rules.txt')

rules_set = pd.read_csv(rules_set_path)
rules_set = rules_set.values

for option in ('train_' + train_ratio, 'valid', 'test'):
    rule_pair_set_path = os.path.join(data_prefix, datasetName, 'train_' + train_round, option + '.csv')
    rules_pair_ids_set = pd.read_csv(rule_pair_set_path, delimiter=",")
    #rules_pair_ids_set
    # split training, validation and testing
    # prepare the rule pairs
    rules_pairs_set = []
    for left_id, right_id, label in rules_pair_ids_set.values:
        rule_left, rule_right = rules_set[left_id][0], rules_set[right_id][0]
        rules_pairs_set.append([rule_left, rule_right, label])
    rules_pairs_set = np.array(rules_pairs_set)
    #rules_pairs_set[:10]
    # save csv
    rules_pairs_df = pd.DataFrame(rules_pairs_set, columns=['rule_left', 'rule_right', 'label'])
    #output_file = os.path.join(data_prefix, datasetName, datasetName + '_rule_pairs_' + option + '.csv')
    output_file = os.path.join(data_prefix, datasetName, 'train_' + train_round, 'rule_pairs_' + option + '.csv')
    rules_pairs_df.to_csv(output_file, index=False)

In [5]:
from datasets import load_dataset

#dataset = load_dataset('csv', data_files='./top-k/rules/test_rules.csv')
_datasets = []
for option in ('train', 'valid', 'test'):
    rule_pairs_file = os.path.join(data_prefix, datasetName, 'train_' + train_round, 'rule_pairs_' + option + '.csv')
    tmp_dataset = load_dataset('csv', data_files=rule_pairs_file)
    _datasets.append(tmp_dataset)




Using custom data configuration default-65988205d4f8e104
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-65988205d4f8e104/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23)
Using custom data configuration default-fcc043272146ee13
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-fcc043272146ee13/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23)
Using custom data configuration default-02868ac1c48c75dc
Reusing dataset csv (/home/user/.cache/huggingface/datasets/csv/default-02868ac1c48c75dc/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23)


In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = cudaOption #'2'
os.environ["WANDB_DISABLED"] = "true"

In [7]:
train_dataset, valid_dataset, test_dataset = _datasets

dataset = train_dataset
dataset['validation'] = valid_dataset['train']
dataset['test'] = test_dataset['train']
# dataset['train'] = dataset['train']
dataset
      


DatasetDict({
    train: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 479
    })
    validation: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 159
    })
    test: Dataset({
        features: ['rule_left', 'rule_right', 'label'],
        num_rows: 161
    })
})

In [8]:
from transformers import AutoTokenizer, BertConfig
batch_size = 16

bertConfig = BertConfig()

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', config=bertConfig, use_fast=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [9]:

rule1_key, rule2_key = 'rule_left', 'rule_right'


def preprocess_function(examples):
    return tokenizer(examples[rule1_key], examples[rule2_key], truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)



Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-65988205d4f8e104/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23/cache-d1d5836883633c8f.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/csv/default-fcc043272146ee13/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23/cache-81f0d95398ee0e71.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
from distutils.command.config import config
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_label = 2
model = AutoModelForSequenceClassification.from_config(config=bertConfig)

In [11]:
metric = 'accuracy'

args = TrainingArguments(output_dir=saved_model_path,
                        evaluation_strategy='epoch',
                        save_strategy='epoch',
                        learning_rate=2e-5,
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        num_train_epochs=20,
                        weight_decay=0.01,
                        load_best_model_at_end=True,
                        metric_for_best_model='accuracy',
                        push_to_hub=False)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
from sklearn import metrics
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #print(predictions, labels)
    precision = metrics.precision_score(labels, predictions)
    recall = metrics.recall_score(labels, predictions)
    f1 = metrics.f1_score(labels, predictions)
    acc = metrics.accuracy_score(labels, predictions)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': acc}


trainer = Trainer(model, args, train_dataset=encoded_dataset['train'], eval_dataset=encoded_dataset['validation'], tokenizer=tokenizer, compute_metrics=compute_metrics)


In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running training *****
  Num examples = 479
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 150


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.688976,0.000000,0.000000,0.000000,0.547170
2,No log,0.721834,0.000000,0.000000,0.000000,0.547170
3,No log,0.690130,0.000000,0.000000,0.000000,0.547170
4,No log,0.692433,0.000000,0.000000,0.000000,0.547170
5,No log,0.692037,0.488095,0.569444,0.525641,0.534591


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16
/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./saved_models/airports-transformer-bc/checkpoint-30
Configuration saved in ./saved_models/airports-transformer-bc/checkpoint-30/config.json
Model weights saved in ./saved_models/airports-transformer-bc/checkpoint-30/pytorch_model.bin
tokenizer config file saved in ./saved_models/airports-transformer-bc/checkpoint-30/tokenizer_config.json
Special tokens file saved in ./saved_models/airports-transformer-bc/checkpoint-30/special_to

TrainOutput(global_step=150, training_loss=0.7088270060221354, metrics={'train_runtime': 35.3496, 'train_samples_per_second': 67.752, 'train_steps_per_second': 4.243, 'total_flos': 160856438138040.0, 'train_loss': 0.7088270060221354, 'epoch': 5.0})

In [15]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 159
  Batch size = 16


/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.6889760494232178,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.5471698113207547,
 'eval_runtime': 0.4592,
 'eval_samples_per_second': 346.241,
 'eval_steps_per_second': 21.776,
 'epoch': 5.0}

In [16]:
ll = trainer.evaluate(encoded_dataset['test'])

f = open(result_file, 'w')
f.write(str(ll))
f.close()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: rule_left, rule_right.
***** Running Evaluation *****
  Num examples = 161
  Batch size = 16
/home/user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.6872183680534363,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.5590062111801242,
 'eval_runtime': 0.4909,
 'eval_samples_per_second': 327.963,
 'eval_steps_per_second': 22.407,
 'epoch': 5.0}

In [17]:
trainer.save_model()

Saving model checkpoint to ./saved_models/airports-transformer-bc
Configuration saved in ./saved_models/airports-transformer-bc/config.json
Model weights saved in ./saved_models/airports-transformer-bc/pytorch_model.bin
tokenizer config file saved in ./saved_models/airports-transformer-bc/tokenizer_config.json
Special tokens file saved in ./saved_models/airports-transformer-bc/special_tokens_map.json
